# MAAP Benchmark with "get-dem" 

## Setting up environment

In [1]:
import os
from time import time
import subprocess
#Uncomment to install numpy
! pip install numpy
import numpy as np
from datetime import datetime
import json
import time
import glob
import shutil

# Uncomment to install sardem
#!pip install sardem
os.environ['PROJ_LIB']='/opt/conda/share/proj'

## Setup test

In [2]:
bboxes = [
    '-118.06817 34.22169 -118.05801 34.22822',  # Mt Wilson (near JPL) (very small, ~8 sec)
    '-124.81360 32.44506 -113.75989 42.24498', # CA and NV (uses ~5-6 GB RAM, takes a 9-10 minutes to run)
    '6.26868 36.00380   18.57179 47.28139'  # Italy (Requires more than 8 GB RAM)
         ]



bbox_index = 0

## ADE Benchmark

Launching the "get-dem" multiple time. 
* Make sure that there are no errors in the "compute" part when running the command line *

In [ ]:
exec_time_list = []
start = time()

i=0
while i<mc_count:
    start = time()
    !sh /projects/get-dem/driver/run.sh -b {bboxes[bbox_index]} --compute TRUE 2>&1 > /dev/null
    exec_time_list.append(time()-start)
    !rm /projects/data/output/dem.tif
    i = i+1


In [ ]:
print("Mean execution time :"+ str(np.mean(exec_time_list)))
print("Mean execution time :"+ str(np.std(exec_time_list)))

## WPST Benchmark

In [3]:
# Setup WPS-T tools
from maap_utils.maap_process import MaapWPST
from maap_utils.maap_authenticator import MaapAuthenticator

maap_auth = MaapAuthenticator('/projects/.maap/auth.ini', '/projects/.maap/maap.ini')
oauth_token = maap_auth.get_esa_token_with_esa_cred()

maap_wpst = MaapWPST('/projects/.maap/maap.ini', oauth_token)

In [17]:
# Generates 
run_id = datetime.now().strftime('%d%m%Y%H%M')
mc_count = 20
print(run_id)
#shutil.rmtree('/projects/data')

170420241452


In [18]:
remaining_job = list(range(mc_count))
launched_job = []
dps_name = []


while len(remaining_job)!=0:
    
    job_num = remaining_job.pop(0)

    maap_job = maap_wpst.launch_process(
        'wf-get-dem',
        [{"id":"s_bbox","data":bboxes[bbox_index],"href":""},
         {"id":"s_compute","data":"TRUE","href":""},
         {'id': 'copy_dir_or_file', 'data': 'dir', 'href': ''},
         {'id': 's3_destination', 'data': '', 'href': 'maap-scientific-data/shared/adu/todelete/benchmark/getdem'+run_id+'_'+str(job_num)},
         {'id': 'source_folder', 'data': '/projects/data/output', 'href': ''}]
    )
    
    if maap_job is not None:
        launched_job.append( (job_num,maap_job) ) 
        dps_name.append (maap_job.dps_name)
    else:
        launched_job.append(job_num)
        
    time.sleep(1)
    
while len(launched_job)!=0:
        
    (job_num,a_job) = launched_job.pop(0)
    maap_wpst.wait_for_final_status(a_job)
    maap_wpst.write_outputs(a_job,'/projects/data/'+run_id+'_'+str(job_num))
    maap_wpst.delete_job(a_job)
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
{'jobId': '661fe2168e067a372cb6b013', 'status': 'ACCEPTED', 'message': 'https://argo.int.esa-maap.org/workflows/argo/exec-wf-get-dem-2024-04-17-14-52-05-92', 'progress': None}
{'jobId': '661fe2178e067a372cb6b015', 'status': 'ACCEPTED', 'message': 'https://argo.int.esa-maap.org/workflows/argo/exec-wf-get-dem-2024-04-17-14-52-07-18', 'progress': None}
{'jobId': '661fe2188e067a372cb6b017', 'status': 'ACCEPTED', 'message': 'https://argo.int.esa-maap.org/workflows/argo/exec-wf-get-dem-2024-04-17-14-52-08-36', 'progress': None}
{'jobId': '661fe2198e067a372cb6b019', 'status': 'ACCEPTED', 'message': 'https://argo.int.esa-maap.org/workflows/argo/exec-wf-get-dem-2024-04-17-14-52-09-58', 'progress': None}
{'jobId': '661fe21a8e067a372cb6b01b', 'status': 'ACCEPTED', 'message': 'https://argo.int.esa-maap.org/workflows/argo/exec-wf-get-dem-2024-04-17-14-52-10-77', 'progress': None}
{'jobId': '661fe21b8e067a372cb6b01d', 'status': '

## Collect metrics

In [19]:
# Collect profile.json files
profile_glob = '/projects/data/*/profile.json'
profile_json_path_list = glob.glob(profile_glob);
#print(profile_json_path_list)


In [20]:

# get-dem ellapsed time
elapsed_time = []
# Maximum RAM
max_footprint_mb = []

# Metrics for line of code where n_cpu_percent_c!=0
relevant_loc = {}

for profile_json_path in profile_json_path_list :
    with open(profile_json_path) as f:
        profile_json= json.load(f)
        
        elapsed_time.append(float(profile_json['elapsed_time_sec']))
        max_footprint_mb.append(float(profile_json['max_footprint_mb']))
        
        for code_line in profile_json["files"]["/opt/get-dem/get_dem.py"]["lines"]:
    
            if code_line["n_cpu_percent_c"] != 0.0 :
            
                if code_line['lineno'] not in relevant_loc:
                    relevant_loc[code_line['lineno']] = {"line":code_line['line'],'n_core_utilization':[],'n_peak_mb':[]}
                    
                relevant_loc[code_line['lineno']]['n_core_utilization'].append(float(code_line['n_core_utilization']))
                relevant_loc[code_line['lineno']]['n_peak_mb'].append(float(code_line['n_peak_mb']))

        
        

In [21]:
print( "Average elapsed time : {:.2f} sec".format(np.mean(elapsed_time)))
print( "Average maximum RAM footprint  : {:.2f} Mb".format(np.mean(max_footprint_mb)))
 

Average elapsed time : 1.50 sec
Average maximum RAM footprint  : 11.21 Mb


In [22]:
print("===== Relevant Line-Of-Code metrics =====\n")
for met_loc in relevant_loc:
    print( "line {} : {}".format(met_loc,relevant_loc[met_loc]["line"]))
    print( "\tAverage n_core_utilization : {:.2f} sec".format(np.mean(relevant_loc[met_loc]["n_core_utilization"])))
    print( "\tAverage n_peak_mb : {:.2f} Mo".format(np.mean(relevant_loc[met_loc]["n_peak_mb"])))
    print(" \n ----- \n")

===== Relevant Line-Of-Code metrics =====

line 8 : import numpy as np

	Average n_core_utilization : 0.07 sec
	Average n_peak_mb : 0.00 Mo
 
 ----- 

line 9 : from osgeo import gdal  # type: ignore

	Average n_core_utilization : 0.06 sec
	Average n_peak_mb : 0.00 Mo
 
 ----- 

line 10 : from sardem import cop_dem  # type: ignore

	Average n_core_utilization : 0.07 sec
	Average n_peak_mb : 10.01 Mo
 
 ----- 

line 66 :     cop_dem.download_and_stitch(dem_file, bbox, output_format="GTiff")

	Average n_core_utilization : 0.05 sec
	Average n_peak_mb : 1.00 Mo
 
 ----- 

line 129 :     parser = argparse.ArgumentParser()

	Average n_core_utilization : 0.04 sec
	Average n_peak_mb : 0.00 Mo
 
 ----- 

line 27 : class Args:

	Average n_core_utilization : 0.05 sec
	Average n_peak_mb : 0.00 Mo
 
 ----- 



In [23]:
dps_elapsed_s = []
dps_cpu_percent = []
dps_ram_mb = []

for monit in maap_wpst.get_monitoring():
    if monit['processorName'] in dps_name:
        dps_elapsed_s.append(float(monit['executionTime']))
        dps_cpu_percent.append(float(monit['cpuConsumed']))
        dps_ram_mb.append(float(monit['ramConsumed']))

        
print( "DPS Average elapsed time : {:.2f} sec".format(np.mean(dps_elapsed_s)))
print( "DPS Average maximum RAM footprint  : {:.2f} Mb".format(np.mean(dps_ram_mb)))
print( "DPS Average CPU  : {:.2f} %".format(np.mean(dps_cpu_percent)))

DPS Average elapsed time : 130.40 sec
DPS Average maximum RAM footprint  : 284.22 Mb
DPS Average CPU  : 0.07 %
